In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import seaborn as sns
import xarray as xr
import dask
from dask.distributed import Client
import dask.array as da
from pathlib import Path
!pip install cloudpathlib
from cloudpathlib import GSPath
import cdsapi
import cartopy
import glob
import geopandas as gpd
from collections import defaultdict
import rioxarray
from datetime import datetime, timedelta
from scipy.stats import percentileofscore
import time
import rioxarray as rxr
from shapely.geometry import box, Polygon, Point
from rasterio.features import geometry_mask, geometry_window
from multiprocessing import Pool
from functools import partial  # Import partial
from concurrent.futures import ProcessPoolExecutor
from scipy.stats import genextreme
import xarray as xr
import gcsfs  # For interacting with Google Cloud Storage
import tempfile
import shutil
from tempfile import TemporaryDirectory
from google.cloud import storage
import sys, pathlib
sys.path.insert(0, "src")
from ERA5_Preprocess import *

# Input Files

In [ ]:
# Define your bucket and paths
bucket = 'gs://leap-persistent/adamnayak/flood-insurance'
county_shapefile = '../../Local_Data/Geospatial/tl_2019_us_county.shp'
precipitation_nc_path = 'ERA5/Daily_Annual/era5_daily_precipitation_2020.nc'
weights_output_path = 'ERA5/ERA5_county_pixel_weights.npy'

# Determine County Weights

In [ ]:
# Call the function
weights = calculate_pixel_weights_nofrac(
    county_shapefile=county_shapefile,
    precipitation_nc_path=precipitation_nc_path,
    output_path=weights_output_path,
    gcs_bucket=bucket
)

# Aggregate Precipitation

In [ ]:
# Define your paths
precipitation_dir = 'ERA5/Daily_Annual/'
output_dir = 'ERA5/County_Daily_Precip_By_Year'

In [ ]:
# Call the function
calculate_daily_precipitation(
    county_shapefile=county_shapefile,
    precipitation_dir=precipitation_dir,
    weights=weights,
    output_dir=output_dir,
    gcs_bucket=bucket
)

# Rechunk to County

In [ ]:
# Define GCS bucket and paths
input_path = 'ERA5/County_Daily_Precip_By_Year'
output_path = 'ERA5/County_Daily_Precip_By_County'

# Define the range of years to process
precip_years = range(1940, 2024)

In [ ]:
# Call the function
rechunk_precipitation_data_by_county_gcs_efficient(
    years=precip_years,
    input_path=input_path,
    output_path=output_path,
    gcs_bucket=bucket
)

# Calculate County Precip Stats of Interest

In [ ]:
# GCS bucket and folder details
bucket_name = 'leap-persistent'
source_folder = 'adamnayak/flood-insurance/ERA5/County_Daily_Precip_By_County'
local_folder = 'ERA5_County_Precip_Local'

In [ ]:
# Download data from GCS
download_from_gcs(bucket_name, source_folder, local_folder)

In [ ]:
# Process all precipitation data
process_all_precipitation_data_era5(local_folder)

# Split Claims Data

In [ ]:
# Define GCS bucket and paths
claims_path = 'NFIP/State_Policy_Data/FimaNfipClaims.csv'
output_path = 'NFIP/Claims_By_County'

In [ ]:
# Split claims data by county
split_claims_data_by_county_gcs(
    claims_path=claims_path,
    gcs_bucket=bucket,
    output_path=output_path
)

In [ ]:
# GCS bucket and folder details
bucket_name = 'leap-persistent'
source_folder = 'adamnayak/flood-insurance/NFIP/Split_Claims'
local_folder = 'NFIP_Claims_By_County'

# Download data from GCS
download_from_gcs(bucket_name, source_folder, local_folder)

# Merge Claims with Precip

In [ ]:
# Define folders for analysis
claims_folder = 'NFIP_Claims_By_County'
precip_folder = 'ERA5_Daily_Precip_Processed_County'
output_folder = 'ERA5_Merged_Claims'

In [ ]:
# Merge claims with precipitation data
merge_claims_with_precip(claims_folder=claims_folder, 
                         precip_folder=precip_folder, 
                         output_folder=output_folder)

# Concatenate Claims with Precip

In [ ]:
processed_path="ERA5_Merged_Claims"
output_file="ERA5_Processed_Claims.csv"

In [ ]:
# Concatenate all processed claims
concatenate_processed_claims(output_file=output_file, 
                                 processed_path=processed_path)

# Clean Up and Push to Cloud

In [ ]:
# Define bucket and folder paths
bucket_name = "leap-persistent"
target_folder = "adamnayak/flood-insurance/ERA5"  # Updated path

In [ ]:
local_paths = ['ERA5_Ann_Max_Precip_County', 'ERA5_Daily_Precip_Processed_County', 'ERA5_Merged_Claims', 'ERA5_Processed_Claims.csv']
remove_flags = [True, True, False, False]  # Specify which items to remove after upload

push_to_cloud_and_conditional_remove(local_paths, remove_flags, bucket_name, target_folder)

In [ ]:
!rm -r ERA5_County_Precip_Local
!rm -r NFIP_Claims_By_County
# Note that we want to keep ERA5_Merged_Claims folder until all datasets are merged to the claims